# Taller de búsqueda de hiperparámetros

En este taller vamos a explorar la búsqueda de hiperparámetros de manera automática.

In [ ]:
import time
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
X, y = load_breast_cancer(return_X_y=True)

In [ ]:
test_size=0.25
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

**Ejemplo 1:** encuentra los mejores hiperparámetros usando `GridSearchCV`

In [ ]:
clf = DecisionTreeClassifier()

# Define los valores que usarás en la búsqueda de hiperparámetros
# max_depth: profundidad máxima del árbol de decisión
# min_samples_leaf: número mínimo de muestras requeridas en una hoja
# criterion: función para medir la calidad de una división
param_grid = {
    "max_depth": (3, 6, 12, 18),
    "min_samples_leaf": (1, 2, 3),
    "criterion": ["gini", "entropy"]
}

# Utiliza GridSearchCV para realizar una búsqueda exhaustiva de hiperparámetros
# estimator: el modelo a usar (DecisionTreeClassifier)
# param_grid: diccionario con los parámetros a probar
# cv=5: validación cruzada con 5 pliegues
# scoring='accuracy': métrica de evaluación (exactitud)
# n_jobs=-1: usar todos los procesadores disponibles para acelerar la búsqueda
gs = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

t0 = time.time()
gs.fit(X_train, y_train)
print("Tiempo de búsqueda: {:.3f}s".format(time.time() - t0))

Podemos ver que el atributo `cv_results_` nos entrega los resultados de toda la búsqueda.

Este atributo contiene un diccionario con información detallada sobre todas las combinaciones de parámetros probadas, incluyendo:
- Los parámetros utilizados en cada iteración
- Los puntajes de validación cruzada obtenidos
- Los tiempos de ajuste y puntuación
- Las desviaciones estándar de los puntajes

In [ ]:
dir(gs)

In [ ]:
gs.cv_results_

Lo más importante es extraer los hiperparámetros del modelo que obtuvieron el mejor puntaje en validación cruzada.

El atributo `best_params_` devuelve un diccionario con la combinación de hiperparámetros que logró el mejor rendimiento durante la búsqueda. Estos son los parámetros que deberías usar para entrenar tu modelo final.

In [ ]:
gs.best_params_

También es posible obtener el mejor resultado en la métrica usada.

El atributo `best_score_` devuelve el puntaje promedio de validación cruzada obtenido por la mejor combinación de hiperparámetros. Este valor te da una estimación del rendimiento esperado del modelo con estos parámetros.

In [ ]:
gs.best_score_

Finalmente, es posible extraer directamente un estimador que ha sido creado con los mejores hiperparámetros.

El atributo `best_estimator_` devuelve una instancia del modelo ya entrenado con los mejores parámetros encontrados. Este modelo está listo para usar en predicciones sin necesidad de entrenamiento adicional.

In [ ]:
gs.best_estimator_

**Ejemplo 2:** la clase `RandomizedSearchCV` se puede usar casi de la misma manera, solo que esta vez se debe escoger un número de combinaciones a evaluar; las cuales se escogerán de manera aleatoria.

### Ventajas de RandomizedSearchCV:
- **Eficiencia:** No prueba todas las combinaciones posibles, solo un subconjunto aleatorio
- **Escalabilidad:** Funciona mejor cuando hay muchos hiperparámetros
- **Control de tiempo:** Puedes limitar el número de iteraciones según tu presupuesto de tiempo
- **Distribuciones continuas:** Puede muestrear de distribuciones continuas (no solo valores discretos)

### Cuándo usar cada método:
- **GridSearchCV:** Cuando tienes pocos hiperparámetros y quieres una búsqueda exhaustiva
- **RandomizedSearchCV:** Cuando tienes muchos hiperparámetros o un presupuesto de tiempo limitado

In [ ]:
clf = DecisionTreeClassifier()

# Define los valores que usarás en la búsqueda de hiperparámetros
# param_dist: distribución de parámetros para el muestreo aleatorio
# max_depth: profundidad máxima del árbol de decisión
# min_samples_leaf: número mínimo de muestras requeridas en una hoja
# criterion: función para medir la calidad de una división
param_dist = {
    "max_depth": (3, 6, 12, 18),
    "min_samples_leaf": (1, 2, 3),
    "criterion": ["gini", "entropy"]
}

# Utiliza RandomizedSearchCV para realizar una búsqueda aleatoria de hiperparámetros
# estimator: el modelo a usar (DecisionTreeClassifier)
# param_distributions: diccionario con las distribuciones de parámetros a muestrear
# n_iter: número de combinaciones de parámetros a probar (10 en este caso)
# cv=5: validación cruzada con 5 pliegues
# scoring='accuracy': métrica de evaluación (exactitud)
# n_jobs=-1: usar todos los procesadores disponibles para acelerar la búsqueda
# random_state=42: semilla para reproducibilidad de los resultados
rs = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

t0 = time.time()
rs.fit(X_train, y_train)
print("Tiempo de búsqueda: {:.3f}s".format(time.time() - t0))

In [ ]:
rs.best_score_

Podemos ver que aunque se demoró mucho menos, el resultado no es tan bueno.

### Comparación de resultados:
- **GridSearchCV:** Busca exhaustivamente, garantiza encontrar la mejor combinación dentro del espacio definido
- **RandomizedSearchCV:** Busca aleatoriamente, puede encontrar buenas combinaciones más rápidamente

### Recomendaciones:
1. **Comienza con RandomizedSearchCV** para explorar rápidamente el espacio de hiperparámetros
2. **Usa GridSearchCV** para hacer una búsqueda más fina alrededor de las mejores regiones encontradas
3. **Considera el tiempo disponible** y el número de hiperparámetros al elegir el método
4. **Prueba diferentes valores de n_iter** en RandomizedSearchCV para encontrar el equilibrio entre tiempo y calidad

In [ ]:
# Evaluación de los modelos optimizados en el conjunto de prueba

# Predicciones con GridSearchCV
gs_predictions = gs.best_estimator_.predict(X_test)
gs_accuracy = accuracy_score(y_test, gs_predictions)
gs_f1 = f1_score(y_test, gs_predictions)

# Predicciones con RandomizedSearchCV
rs_predictions = rs.best_estimator_.predict(X_test)
rs_accuracy = accuracy_score(y_test, rs_predictions)
rs_f1 = f1_score(y_test, rs_predictions)

print("=== COMPARACIÓN DE RESULTADOS ===")
print(f"GridSearchCV:")
print(f"  - Mejores parámetros: {gs.best_params_}")
print(f"  - Puntaje en validación cruzada: {gs.best_score_:.4f}")
print(f"  - Exactitud en conjunto de prueba: {gs_accuracy:.4f}")
print(f"  - F1-score en conjunto de prueba: {gs_f1:.4f}")
print()
print(f"RandomizedSearchCV:")
print(f"  - Mejores parámetros: {rs.best_params_}")
print(f"  - Puntaje en validación cruzada: {rs.best_score_:.4f}")
print(f"  - Exactitud en conjunto de prueba: {rs_accuracy:.4f}")
print(f"  - F1-score en conjunto de prueba: {rs_f1:.4f}")
print()

# Comparación directa
print("=== COMPARACIÓN DIRECTA ===")
if gs_accuracy > rs_accuracy:
    print(f"GridSearchCV obtuvo mejor exactitud (+{gs_accuracy - rs_accuracy:.4f})")
else:
    print(f"RandomizedSearchCV obtuvo mejor exactitud (+{rs_accuracy - gs_accuracy:.4f})")

if gs_f1 > rs_f1:
    print(f"GridSearchCV obtuvo mejor F1-score (+{gs_f1 - rs_f1:.4f})")
else:
    print(f"RandomizedSearchCV obtuvo mejor F1-score (+{rs_f1 - gs_f1:.4f})")
